In [1]:
import numpy as np
import lysis
import os

import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.animation import FuncAnimation
from matplotlib.colors import ListedColormap

from IPython.display import HTML
plt.rcParams["animation.html"] = "jshtml"

In [2]:
e = lysis.util.Experiment(os.path.join("..", "..", "data"), experiment_code="2023-01-18-1700")
e.read_file()
file_code = ".f-normal.dat"

In [3]:
print(e)

experiment_code : 2023-01-18-1700
data_filenames  : unbinding_time    : tsectPA.dat
                  lysis_time        : lysismat.dat
                  total_lyses       : lenlysisvect.dat
                  degradation_state : deg.dat
micro_params    : None
macro_params    : binding_rate       : 0.1
                  pore_size          : 0.00010135
                  diffusion_coeff    : 5e-07
                  binding_sites      : 427
                  forced_unbind      : 0.0852
                  average_bind_time  : 27.8
                  grid_node_distance : 1.0862
                  cols               : 9
                  rows               : 12
                  full_row           : 26
                  xz_row             : 17
                  total_edges        : 303
                  total_fibers       : 225
                  empty_rows         : 3
                  last_empty_edge    : 77
                  total_molecules    : 430
                  moving_probability : 0.2
  

In [4]:
deg = np.fromfile(os.path.join(e.os_path, "f_deg_time" + file_code))
tsave = np.fromfile(os.path.join(e.os_path, "tsave" + file_code))

In [5]:
deg = deg.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves, e.macro_params.total_edges)

In [6]:
tsave = tsave.reshape(e.macro_params.total_trials, e.macro_params.number_of_saves)

In [7]:
mapped_deg = deg
mapped_deg[:,:,e.macro_params.last_empty_edge+1:][deg[:,:,e.macro_params.last_empty_edge+1:] == 0] = e.macro_params.total_time * 10
#mapped_deg = -deg
#mapped_deg[deg == 0] = e.macro_params.total_time * 2
#mapped_deg[deg == -1] = 0
#for i in range(e.macro_params.total_edges):
#    mapped_deg[:, :, i] -= tsave

In [10]:
def plot_coords(i, j):
    x = j
    y = i
    if j % 3 == 0:
        return x / 3.0, y + 0.5
    if j % 3 == 1:
        return (x - 1) / 3.0, y
    if j % 3 == 2:
        return (x - 2) / 3.0 + 0.5, y

In [11]:
x = np.empty(e.macro_params.total_edges, dtype=float)
y = np.empty(e.macro_params.total_edges, dtype=float)
for k in range(e.macro_params.total_edges):
    i, j = lysis.from_fortran_edge_index(k, e.macro_params.rows, e.macro_params.cols)
    x[k], y[k] = plot_coords(i, j)

In [12]:
colormap = plt.cm.viridis(np.arange(plt.cm.viridis.N))
colormap[:,3] = 0.75
colormap = ListedColormap(colormap)

#colormap = mpl.colormaps['viridis']
#colormap[:,0:3] *= 0.5
colormap.set_extremes(bad='k', under='w', over='k')

In [20]:
run = 0

plt.ioff()
fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0, 0, 1, 1])#, frameon=False)
vmin = 0
vmax = e.macro_params.cols - 1

ydiff = e.macro_params.rows - e.macro_params.cols

ax.set_xlim(vmin - 0.25, vmax + 0.25)
ax.set_ylim(vmin - 0.25, vmax + ydiff + 0.25)
ax.set_aspect('equal')
fig.canvas.draw()
s = ((ax.get_window_extent().width  / (vmax-vmin+0.5) * 72/(2*fig.dpi)) ** 2)
scatt = ax.scatter(x, y, 
                   s=s, marker='s', linewidths=0,
                   c=mapped_deg[run, 0], cmap=colormap,
                   #vmin = 0, vmax=e.macro_params.total_time,
                   plotnonfinite=True,
                   vmin=1, vmax=mapped_deg[run, e.macro_params.number_of_saves-1][mapped_deg[run, e.macro_params.number_of_saves-1] < e.macro_params.total_time].max())

def update(frame_number):
    scatt.set_array(mapped_deg[run, frame_number])
    
animation = FuncAnimation(fig, update, frames=np.arange(e.macro_params.number_of_saves), interval=200)

#HTML(animation.to_jshtml())
HTML(animation.to_html5_video())